# "COVID-19 por comuna"
> Nuevos casos confirmados por comuna.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [1]:
#hide
import numpy as np
import pandas as pd
import altair as alt

# Nuevos casos confirmados por comuna en la R.M

In [2]:
#hide
county_cumulative_cases = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv")

In [3]:
#hide
county_cumulative_cases_RM = county_cumulative_cases.query("Region=='Metropolitana'").drop(
    columns=["Region", "Codigo region", "Poblacion", "Tasa", "Codigo comuna"])

In [4]:
#hide
county_cases_RM = county_cumulative_cases_RM.set_index("Comuna").T.diff().dropna()

In [5]:
#hide
county_cases_RM["Total"] = county_cases_RM.sum(axis=1)
county_cases_RM = county_cases_RM.T.apply(lambda x: x/x[-1]).T

In [6]:
#hide
start_date = county_cases_RM.reset_index()["index"].iloc[0]
end_date = county_cases_RM.reset_index()["index"].iloc[-1]

data = pd.DataFrame()
data['Fecha'] = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d')
data = data.set_index("Fecha")

In [7]:
#hide
data[list(county_cases_RM.columns)] = county_cases_RM[list(county_cases_RM.columns)]

In [8]:
#hide
data = data.interpolate(method="linear")

In [9]:
#hide
last_known_proportion = data.iloc[-1]

In [10]:
#hide
last_known_date = county_cases_RM.index[-1]
last_known_date

'2020-06-08'

In [11]:
#hide
regional_cumulative_cases = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv')

In [12]:
#hide
regional_cumulative_cases_RM = regional_cumulative_cases.query("Region=='Metropolitana'").iloc[-1].drop("Region")
regional_cases_RM = regional_cumulative_cases_RM.diff().dropna()

In [13]:
#hide
n_missing_days = (pd.to_datetime(regional_cases_RM.index[-1])-pd.to_datetime(data.index[-1])).days

In [14]:
#hide
data = data.T

In [15]:
#hide
for i in np.arange(n_missing_days):
    idx = (pd.to_datetime(last_known_date)+pd.DateOffset(1+i)).strftime("%Y-%m-%d")
    print(idx)
    data[idx] = last_known_proportion

2020-06-09
2020-06-10


In [16]:
#hide
data = data.T

In [17]:
#hide
data["Regional"] = regional_cases_RM

In [18]:
#hide
data = data.T.apply(lambda x: x[:-1]*x[-1]).T.astype(int)

In [19]:
#hide
data_long_form = data.drop(columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Casos confirmados")

In [20]:
#hide_input
input_dropdown = alt.binding_select(options=data_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='category20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(data_long_form).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Casos confirmados", axis=alt.Axis(title="nuevos casos confirmados")),
    tooltip = ["Fecha", "Comuna", "Casos confirmados"],
    color = color
).properties(
    title="Nuevos casos confirmados por comuna en la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19), [Ministerio de Salud](https://www.minsal.cl/)

* No hay datos diarios de nuevos confirmados por comuna. Estos son estimados a través de una interpolación lineal entre las proporciones observadas entre los dos informes más cercanos en el tiempo y aplicados a casos confirmados regionales.

# Advertencia

El número de casos confirmados/informados no representa exactamente el número de casos/contagios con Covid-19 reales. Estos últimos valores no son conocidos por límites de testeo.